In [1]:
import os
from pyarabic.araby import strip_harakat , strip_tashkeel, strip_diacritics,strip_tatweel,sentence_tokenize
import re
import string
import grapheme
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn


In [2]:
import importlib

import utils
import tokenizer

importlib.reload(utils)
importlib.reload(tokenizer)

<module 'tokenizer' from 'd:\\arabic_tashkeela\\tokenizer.py'>

In [3]:
with open("dataset/إتحاف المهرة لابن حجر.txt") as file:
    lines = file.readlines()
data1 = lines[3:]

with open("dataset/أحكام القرآن لابن العربي.txt") as file:
    lines = file.readlines()
data2 = lines[6:]

with open("dataset/إتحاف المهرة لابن حجر.txt") as file:
    lines = file.readlines()
data3 = lines

In [4]:
data = [
    *data1,
    *data2,
    *data3
]

In [5]:
let , tash = utils.extract_let_tash(data[0])

In [6]:
data[0]

'حَدِيثٌ كم حم: أَنَّهُ رَأَى رَسُولَ اللَّهِ، صَلَّى اللَّهُ عَلِيهِ وَسَلَّم، عِنْدَ أَحْجَارِ الزَّيْتِ يَسْتَسْقِي مُقَنَّعًا بِكَفَّيْهِ يَدْعُو، هَكَذَا كم فِي الاسْتِسْقَاءِ: حَدَّثَنَا عَلِيُّ بْنُ حَمْشَاذَ، حَدَّثَنَا عُبَيْدُ بْنُ شَرِيكٍ، حَدَّثَنَا يَحْيَى بْنُ بُكَيْرٍ، عَنِ اللَّيْثِ، عَنْ خَالِدِ بْنِ يَزِيدَ، عَنْ سَعِيدِ بْنِ أَبِي هِلالٍ، عَنْ يَزِيدَ بْنِ عَبْدِ اللَّهِ، هُوَ ابْنُ الْهَادِ عَنْ مُحَمَّدِ بْنِ إِبْرَاهِيمَ، عَنْ عُمَيْرٍ، مَوْلَى آبِي اللَّحْمِ، عَنْهُ بِهَذَا، وَقَالَ: صَحِيحُ الإِسْنَادِ، وَفِي الدُّعَاءِ: حَدَّثَنَا أَبُو الْعَبَّاسِ مُحَمَّدُ بْنُ يَعْقُوبَ، حَدَّثَنَا مُحَمَّدُ بْنُ عَبْدِ اللَّهِ بْنِ عَبْدِ الْحَكَمِ، حَدَّثَنَا أَبِي، وَشُعَيْبُ بْنُ اللَّيْثِ، قَالا: حَدَّثَنَا اللَّيْثُ، بِهِ وَرَوَاهُ أَحْمَدُ: عَنْ قُتَيْبَةَ، عَنِ اللَّيْثِ، بِهِ.(1/1)\n'

In [7]:
tash[:10]

['َ', 'ِ', ' ', 'ٌ', ' ', ' ', ' ', ' ', ' ', ' ']

In [8]:
let[:10]

['ح', 'د', 'ي', 'ث', ' ', 'ك', 'م', ' ', 'ح', 'م']

In [9]:
u_let , u_tash = utils.sample_unique(data)

In [10]:
let_tok = tokenizer.Tokenizer(u_let,20)
tash_tok = tokenizer.Tokenizer(u_tash,20)

In [11]:
tash_tok.char2id

{' ': 2,
 'َّ': 3,
 'ُ': 4,
 'َ': 5,
 'ٍ': 6,
 'ُّ': 7,
 'ً': 8,
 'ٌ': 9,
 'ْ': 10,
 'ِ': 11,
 'ِّ': 12,
 '<حشو>': 0,
 '<مجهول>': 1}

In [12]:
let , tash = utils.extract_let_tash(data[0])
let_tokens = let_tok.sent_to_ids(let)
let_chars = let_tok.ids_to_sent(let_tokens)
print(let)
print(let_tokens)
print(let_chars)

['ح', 'د', 'ي', 'ث', ' ', 'ك', 'م', ' ', 'ح', 'م', ' ', 'أ', 'ن', 'ه', ' ', 'ر', 'أ', 'ى', ' ', 'ر', 'س', 'و', 'ل', ' ', 'ا', 'ل', 'ل', 'ه', ' ', 'ص', 'ل', 'ى', ' ', 'ا', 'ل', 'ل', 'ه', ' ', 'ع', 'ل', 'ي', 'ه', ' ', 'و', 'س', 'ل', 'م', ' ', 'ع', 'ن', 'د', ' ', 'أ', 'ح', 'ج', 'ا', 'ر', ' ', 'ا', 'ل', 'ز', 'ي', 'ت', ' ', 'ي', 'س', 'ت', 'س', 'ق', 'ي', ' ', 'م', 'ق', 'ن', 'ع', 'ا', ' ', 'ب', 'ك', 'ف', 'ي', 'ه', ' ', 'ي', 'د', 'ع', 'و', ' ', 'ه', 'ك', 'ذ', 'ا', ' ', 'ك', 'م', ' ', 'ف', 'ي', ' ', 'ا', 'ل', 'ا', 'س', 'ت', 'س', 'ق', 'ا', 'ء', ' ', 'ح', 'د', 'ث', 'ن', 'ا', ' ', 'ع', 'ل', 'ي', ' ', 'ب', 'ن', ' ', 'ح', 'م', 'ش', 'ا', 'ذ', ' ', 'ح', 'د', 'ث', 'ن', 'ا', ' ', 'ع', 'ب', 'ي', 'د', ' ', 'ب', 'ن', ' ', 'ش', 'ر', 'ي', 'ك', ' ', 'ح', 'د', 'ث', 'ن', 'ا', ' ', 'ي', 'ح', 'ي', 'ى', ' ', 'ب', 'ن', ' ', 'ب', 'ك', 'ي', 'ر', ' ', 'ع', 'ن', ' ', 'ا', 'ل', 'ل', 'ي', 'ث', ' ', 'ع', 'ن', ' ', 'خ', 'ا', 'ل', 'د', ' ', 'ب', 'ن', ' ', 'ي', 'ز', 'ي', 'د', ' ', 'ع', 'ن', ' ', 'س', 'ع', 'ي', 'د', ' ', 'ب',

In [24]:
class CustomImageDataset(Dataset):
    def __init__(self, data):
        self.data = random.sample(data,100)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sent = self.data[idx]
        let , tash = utils.extract_let_tash(sent)
        let_tokens = let_tok.sent_to_ids(let)
        tash_tokens = tash_tok.sent_to_ids(tash)

        return torch.tensor(let_tokens), torch.tensor(tash_tokens)


In [25]:
dataset = CustomImageDataset(data)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [26]:
dataset[7]

(tensor([15, 28, 23, 16, 18, 23, 16, 22, 12,  7, 18, 14, 16, 17, 18, 23, 16, 16,
          8, 18]),
 tensor([ 5,  5,  2,  5,  2,  2,  2,  3, 11,  7,  2,  5,  3,  2,  2,  2,  2,  3,
          4,  2]))

In [27]:
len(dataset)

100

In [28]:
# Define Model
class TokenClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(TokenClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        logits = self.fc(lstm_out)
        return logits

In [29]:
vocab_size = len(let_tok.char2id)
embedding_dim = 5
hidden_dim = 10
output_dim = len(tash_tok.char2id)

model = TokenClassifier(vocab_size, embedding_dim, hidden_dim, output_dim)

In [32]:
# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 100

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for  i , (tokens, labels) in enumerate(dataloader):
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(tokens)
        
        # Reshape outputs and labels to (batch_size * seq_length, output_dim)
        outputs = outputs.view(-1, output_dim)
        labels = labels.view(-1)
        
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataloader)}')

Epoch 1/100, Loss: 2.0661516785621643
Epoch 2/100, Loss: 2.0130794048309326
Epoch 3/100, Loss: 2.0072998702526093
Epoch 4/100, Loss: 1.9880936741828918
Epoch 5/100, Loss: 1.9517181515693665
Epoch 6/100, Loss: 1.9216060638427734
Epoch 7/100, Loss: 1.905726134777069
Epoch 8/100, Loss: 1.8711503148078918
Epoch 9/100, Loss: 1.866332471370697
Epoch 10/100, Loss: 1.8483146131038666
Epoch 11/100, Loss: 1.8331486284732819
Epoch 12/100, Loss: 1.8203915059566498
Epoch 13/100, Loss: 1.782901555299759
Epoch 14/100, Loss: 1.7563607394695282
Epoch 15/100, Loss: 1.8233104944229126
Epoch 16/100, Loss: 1.785841315984726
Epoch 17/100, Loss: 1.753622829914093
Epoch 18/100, Loss: 1.7441161572933197
Epoch 19/100, Loss: 1.7414315044879913
Epoch 20/100, Loss: 1.7152418792247772
Epoch 21/100, Loss: 1.716612845659256
Epoch 22/100, Loss: 1.737771600484848
Epoch 23/100, Loss: 1.7547439932823181
Epoch 24/100, Loss: 1.6636933386325836
Epoch 25/100, Loss: 1.7332665026187897
Epoch 26/100, Loss: 1.6853596568107605
Ep

In [66]:
def inference(sent):
    lets , tashs = utils.extract_let_tash(sent)
    let_tokens = let_tok.sent_to_ids(lets)
    let_tokens_tensor = torch.tensor(let_tokens)
    let_tokens_tensor
    with torch.no_grad():
        outs = model(let_tokens_tensor)
        
    max_values, max_indices =torch.max(outs, dim=1)
    tashs_preds = tash_tok.ids_to_sent(max_indices.tolist())
    pred="".join([char+pred_tash if pred_tash!=" " else char for char,pred_tash in list(zip(lets , tashs_preds))])
    return pred

In [67]:
sent = data[600]
inference(sent)

'حدَيث عهَ حب حمَ أَنَ أَخَتَ'